# Shallow Aquifer model

*This notebook runs a simple 1D model for groundwater flow and water table changes in a shallow unconfined aquifer above a horizontal impermeable layer.*

(Author: Greg Tucker, University of Colorado, Boulder. Version 1.0, March 2020)

## How do I run the model?

First, run the complete notebook (Cell => Run All).

Next, page down to the section "Example model run" to see an example of how to run the model and plot its output.

Then, continue on to the section "Your turn" to read about how to change parameters and re-run the model. Use the cells below that section to type your commands.

## What to see and do?

- Watch the movie in the example run. The dashed line shows the expected steady state water table: the water table position that occurs when the recharge entering the aquifer from above is exactly balanced by the outflow along the right-side boundary. Approximately how long does this run take to reach that steady state?

- Experiment with changing the value of hydraulic conductivity ($K$). How does conductivity influence the height of the water table when the system is in steady state? How does it influence the time needed to reach steady state?

- Experiment with changing the value of recharge ($R$). How does recharge influence the height of the water table when the system is in steady state? How does it influence the time needed to reach steady state?

- If you simultaneously double both the recharge and conductivity, does the steady state water table height change? Why or why not?

- Given what you have observed, would you expect the water table in a dry climate (low $R$) to be higher or lower than in a wet climate (high $R$), all else equal?

- If recharge were to suddenly change, which aquifer would respond faster: one with a high conductivity, or one with a low conductivity?

## What's under the hood?

The model using the Dupuit approximation for a "shallow" unconfined aquifer, with the hydraulic gradient equal to the water table gradient. The aquifer sits on top of a horizontal impermeable layer. Darcy's law for this situation can be written:

$q = -K H \frac{\partial H}{\partial x}$. 

Here, $q$ is the water discharge per unit aquifer width, $H$ is the aquifer thickness (and also the height of the water table above the horizontal impermeable layer), and $x$ is distance from the left edge.

Mass balance says that the rate of rise or fall of the water table at a particular position depends on the incoming recharge rate, $R$, minus the derivative of discharge:

$(1-\phi ) \frac{\partial H}{\partial t} = R - \frac{\partial q}{\partial x}$,

where $t$ is time and $\phi$ is porosity.

The model is solved (approximated) numerically using a finite-difference scheme. The left boundary is an impermeable "wall"; the right boundary is a seepage face, meaning that the water table thickness is zero at this edge, and the water percolates out to the right at a rate that depends on the water table slope (hydraulic gradient) and aquifer thickness.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

The basic source code for the model is here. You shouldn't need to modify it, but take a look to a get a sense for how it works.

In [ ]:
class WaterTableSimulator(object):
    
    def __init__(self,
                 K = 100., # hydraulic conductivity (m/day)
                 R = 2e-3, # recharge rate (m/day)
                 H0 = 0.0, # initial height (m)
                 n = 50,   # number of nodes in the model
                 dx = 1.,  # spacing between nodes (m)
                 dt = 1e-4): # time step (days)

        self.hydraulic_conductivity = K
        self.recharge_rate = R
        self.init_height = H0
        self.init_time = 0
        self.nodes = n
        self.dx = dx
        self.dt = dt
        self.length = self.dx * self.nodes
        self.centers = np.arange(0.5*self.dx, self.length+self.dx, self.dx)
        self.H = 0 * self.centers + self.init_height
        self.H[-1] = 0
        self.u = 0 * self.centers
        self.q = 0 * self.centers
        
        # Construct an analytic solution for the steady state water table
        self.Hanalytic = np.sqrt(np.abs((R/K) * ((self.length + self.dx/2)**2 
                                                 - self.centers**2)))
        self.Qanalytic = self.centers * R
        self.Uanalytic = np.zeros(len(self.Qanalytic))
        self.Uanalytic[:-1] = self.Qanalytic[:-1] / self.Hanalytic[:-1]

        self.current_time = 0.0

    def run_one_step(self):
        """Run one time step."""
        self.dH = np.diff(self.H)      # differences in water table height btwn adjacent nodes
        self.dHdx = self.dH / self.dx  # water table gradient (= hydraulic gradient)
        self.u[1:] = -self.hydraulic_conductivity * self.dHdx  # Darcian velocity, m/day
        self.q[1:] = self.u[1:] * (self.H[:-1] + 0.5 * self.dH) # Unit discharge, m2/day
        self.dHdt = self.recharge_rate - np.diff(self.q) / self.dx # Rate of WT rise/fall, m/day
        self.H[:-1] += self.dHdt * self.dt  # New WT height for this time step, m
        self.current_time += self.dt   # Update current time

    def run_n_steps(self, n):
        for _ in range(n):
            self.run_one_step()
        print('Current time = ' + str(self.current_time))


In [ ]:
def run_model(run_duration = 400.0,  # run duration (days)
                 K = 100., # hydraulic conductivity (m/day)
                 R = 2e-3, # recharge rate (m/day)
                 H0 = 0.0, # initial height (m)
                 n = 50,   # number of nodes in the model
                 dx = 1.,  # spacing between nodes (m)
                 dt = 1e-4, # time step (days)
                 plot_interval_in_days = 10,
             ):
    """Initialize, run, and display output from model.

    Parameters
    ----------
    """

    # Instantiate and initialize a simulator
    model = WaterTableSimulator(K=K, R=R, H0=H0, n=n, dx=dx, dt=dt)

    # Calculate number of animation iterations
    save_every = int(plot_interval_in_days / model.dt)
    nsteps = int(run_duration / plot_interval_in_days)

    # Set up a blank figure with placeholder lists for data
    fig, ax = plt.subplots()
    xdata = []
    ydata = []
    obj = ax.plot([], [], color = 'k')

    # Then, set up an initialization function
    def init():
        ax.set_ylim(0, 1.2 * np.amax(model.Hanalytic))
        ax.set_xlim(0, model.length)
        ax.set_ylabel('Height (m)')
        ax.set_xlabel('Distance (m)')
        return(obj)

    # Next, define the update function
    def update(i):
        ax.cla()
        model.run_n_steps(save_every)
        xdata = model.centers
        ydata = model.H
        ax.set_ylim(0, 1.2 * np.amax(model.Hanalytic))
        ax.set_xlim(0, model.length)
        ax.set_ylabel('Height (m)')
        ax.set_xlabel('Distance (m)')
        ax.set_title('Time = ' + str(round(model.current_time)) + ' days')
        obj = ax.plot(xdata, ydata)
        obj = ax.plot(xdata, model.Hanalytic, 'k--')
        return(obj)

    # Run the animation!
    print('Running...')
    anim = FuncAnimation(fig, update, nsteps, init_func=init, blit = True)

    # Convert the animation to HTML
    vid = HTML(anim.to_html5_video())
    
    print('done!')

    return vid, model

## Example model run

Use the syntax below to run the model. This particular example uses default values for the hydraulic conductivity, recharge rate, and other parameters. See "Your Turn" below to learn how to change these parameters.

In [ ]:
vid, model = run_model()
vid

Once you have completed a model run, you can view and plot the water table height (=saturated zone thickness), the Darcian velocity, the unit discharge, and other variables. The variable `centers` is a numpy array containing the $x$ coordinates of the cell centers. Here are some examples:

In [ ]:
plt.plot(model.centers, model.H)
plt.xlabel('Distance (m)')
plt.ylabel('Height (m)')
plt.title('Final water table height')

In [ ]:
plt.plot(model.centers, model.u)
plt.plot(model.centers[:-1], model.Uanalytic[:-1], 'k:')  # note: last node undefined, don't plot
plt.xlabel('Distance (m)')
plt.ylabel('Darcian velocity (m/day)')
plt.title('Final flow velocity')

In [ ]:
plt.plot(model.centers, model.q)
plt.plot(model.centers[:-1], model.Qanalytic[:-1], 'k:')  # note: last node undefined, don't plot
plt.xlabel('Distance (m)')
plt.ylabel(r'Discharge per unit width (m$^2$/day)')
plt.title('Final aquifer discharge')

## Your turn!

Use new cells below to run the model with different values of $K$, $R$, or other inputs. To change from the default value of a parameter, specify the name and value as keyword arguments to the `run_model` function. For example, to run the model with twice the default conductivity (that is, 200 m/day instead of 100 m/day), you would use `vid, model = run_model(K=200.0)`. To play the resulting movie, type `vid` on the last line or in a cell by itself. See the examples above for guidance on how to plot your data.

To get plots into your paper, you have a few options:

(1) Use the `savefig` matplotlib function to save your figures to files, and then import them into the notebook that contains your paper. For example, to save a plot as a file called `myplot.png`, use `savefig('myplot.png')` on a line of code right after your plotting commands.

(2) Copy the model code plus your runs at the end of your notebook, and use a markdown cell to give each figure a number and caption. Then you can refer to it in the main text.

(3) Take screen shots of your figures, and place them in the notebook containing your paper.

Whatever method you use, please remember to number your figures and include captions.

### Enter your work below here: